### USE PYG ENVIRONMENT!!!!

In [ ]:

# # ### USE PYG ENVIRONMENT!!!!
# # !pip install torch


# # %%
# # Install required packages
# import os, torch

# os.environ['TORCH'] = torch.__version__
# !pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
# !pip install git+https://github.com/pyg-team/pytorch_geometric.git

# !pip install sentence_transformers
# !pip3 install fuzzywuzzy[speedup]
# !pip install captum
# !pip install torch-sparse
# !pip install torch-scatter


In [ ]:

import os, torch

### load graph data 
import pickle

with open('../data/graphs/linegraph_tg.pkl', 'rb') as f:
    data = pickle.load(f)

import os, torch

from torch_geometric.data import DataLoader
import torch_geometric.transforms as T
import numpy as np

train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1, 
)(data)


In [ ]:
from torch_geometric.loader import NeighborLoader

# Convert boolean masks to indices
data.train_mask = data.y > 0
train_nodes = data.train_mask.nonzero(as_tuple=False)
batch_size = 1

# Set the input nodes for the loader
loader = NeighborLoader(data, batch_size=batch_size,
                        shuffle=True, 
                        num_neighbors=[-1]*50,
                        input_nodes=train_nodes)  # Use the indices instead of the boolean mask


In [ ]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(80085)
        self.conv1 = GCNConv(data.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin =  Linear(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x


In [ ]:
# !pip3 install seaborn
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
model = GCN(hidden_channels=20)


In [ ]:

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.MSELoss()


In [ ]:
def train():
    model.train()
    optimizer.zero_grad()  # Clear gradients
    losses = []
    first = True
    for batch in loader:
        out = model(batch.x, batch.edge_index)
        loss = criterion(out[0], batch.y[0])
        loss.backward()  # Backward pass.
        optimizer.step()
        losses.append(loss.item())
        if first:
            print(out[0], batch.y[0], loss.item())
            first = False
    return torch.tensor(losses).mean().item()

epochs, losses = [], []
for epoch in range(1, 10000):
    loss = train()
    epochs.append(epoch)
    losses.append(loss)
    # if epoch % 10 == 0:
    print(f'Epoch: {epoch:03d}, Loss: {loss}')
